In [1]:
# set module auto-reloaded 
%load_ext autoreload
%autoreload 2

In [2]:
# Load MNIST data set

from tensorflow.keras.datasets import mnist
train_set, test_set = mnist.load_data()

In [8]:
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
import numpy as np
import dahuffman
import pickle
from Compressible_Huffman import Huffman

# generate a simple NN model

kernel_initializer = 'he_normal'
activation = "relu"

def get_model(chs=128):
    shape = (28, 28, 1)

    inputs = Input(shape)
    layer = Flatten()(inputs)
    layer = Dense(units=chs * 2, activation=activation, kernel_initializer=kernel_initializer)(layer)
    layer = Dense(units=chs, activation=activation, kernel_initializer=kernel_initializer)(layer)
    output = Dense(10, activation='linear', use_bias=True, kernel_initializer=kernel_initializer)(layer)

    model = Model(inputs, output)
    return model
    
# Instantiate the Huffman class and create the original model
model = get_model()  # Replace with your own model creation code
compNN = Huffman(model)
optimizer = tf.optimizers.Adam(learning_rate=1e-3, beta_1=0.9)
compNN.compile(optimizer, loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True))

original_weights = compNN.net_model.get_weights()

# Save the original weights
with open('original_model_weights.pkl', 'wb') as file:
    pickle.dump(original_weights, file)

# compNN.weights
compressed_weights = compNN.compressNN()

# set compNN.weights to 0
for layer in compNN.net_model.layers:
    if isinstance(layer, tf.keras.layers.Dense):
        layer.set_weights([tf.zeros_like(w) for w in layer.get_weights()])
            
# Save the compNN instance to a file
with open('compressed_nn.pkl', 'wb') as outp:  # Overwrites any existing file.
    pickle.dump(compNN.net_model, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(compNN.codec, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(compressed_weights, outp, pickle.HIGHEST_PROTOCOL)

In [6]:
with open('test1.pkl', 'wb') as outp:  # Overwrites any existing file.
    pickle.dump(compNN.net_model, outp, pickle.HIGHEST_PROTOCOL)

In [7]:
with open('test2.pkl', 'wb') as outp:  # Overwrites any existing file.
    pickle.dump(model.net_model, outp, pickle.HIGHEST_PROTOCOL)

AttributeError: 'Functional' object has no attribute 'net_model'